# XBeach profile evolution

Once your model finished running, it is time to have a look at the model netCDF outputs. XBeach generates a single NC file with all data in it. 

First we need to load the IPython libraries:

In [ ]:
%pylab inline
%matplotlib inline
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from JSAnimation import IPython_display

In [ ]:
nc_data = netCDF4.Dataset('./myprofile3/xboutput.nc')
nc_data2 = netCDF4.Dataset('./myprofile4/xboutput.nc')
print nc_data.variables.keys()

To query the type/definition of each of the `netcdf` keys we can do:

In [ ]:
nc_data.variables['H']

# Get visualisation parameters

In [ ]:
# Export the stored time values
times = nc_data.variables['globaltime'][:]

# Export the X-axis values of the profile
xprofile = nc_data.variables['globalx'][0,:]

# Export the bed elevation
bed_elev = nc_data.variables['zb'][:,0,:]
bed_elev2 = nc_data2.variables['zb'][:,0,:]

# Export the wave height
wave_height = nc_data.variables['H'][:,0,:]
wave_height2 = nc_data2.variables['H'][:,0,:]

# Export the water level
water_lvl = nc_data.variables['zs'][:,0,:]
water_lvl2 = nc_data2.variables['zs'][:,0,:]

## Check parameters shape

The shape of an array gives us the dimension of the dataset we are looking at:

In [ ]:
print 'shape time values: ',times.shape
print 'shape X-axis profile: ',xprofile.shape
print 'shape bed elevation: ',bed_elev.shape
print 'shape wave height: ',wave_height.shape
print 'shape water level: ',water_lvl.shape

# Plotting XBeach model at a given time

To plot the model result for a given time we can define a function that will handle the process automatically. To define a function in IPython we use the following keyword: `def`

In [ ]:
# FID is the frame ID: an integer defining the time step we want to plot
def xbeach_frame1(FID):
    
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(10,5), dpi=80) 
    axes = plt.gca()
    
    # Set extend of computational domain
    axes.set_xlim([0,5000])
    axes.set_ylim([-50,20])

    # Define the base of the model
    base = np.zeros(len(xprofile[:]))
    base.fill(-60)
    
    # Define bed elevation for given frame ID
    bed = bed_elev[FID,:]
    ax1.plot(xprofile[:],bed,linewidth=2.0,color=[139./255.,131./255.,120./255.])
    ax1.fill_between(xprofile[:], base, bed, where=bed >= base, facecolor=[1.0,0.9,0.6], interpolate=True)

    # Define top water surface  
    sea = water_lvl[FID,:] + wave_height[FID,:]
    ax1.plot(xprofile[:],sea,linewidth=2.0,color=[0./255.,0./255.,128./255.])
    ax1.fill_between(xprofile[:], bed, sea, where= sea > bed, facecolor=[0.7,0.9,1.0], interpolate=True)

    # Axes definitions
    tlt='XBeach model 1 time:'+str(int(times[FID]))+' s'
    plt.title(tlt, fontsize=12)
    ax1.set_ylabel('elevation [m]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    
    # Set legend
    names=['zb','H+zs']
    ax1.legend(names,loc='lower right', fontsize=11)
    plt.show()
    
def xbeach_frame2(FID):
    
    # Set figure size
    fig, ax1 = plt.subplots(figsize=(10,5), dpi=80) 
    axes = plt.gca()
    
    # Set extend of computational domain
    axes.set_xlim([0,5000])
    axes.set_ylim([-50,20])

    # Define the base of the model
    base = np.zeros(len(xprofile[:]))
    base.fill(-60)
    
    # Define bed elevation for given frame ID
    bed = bed_elev2[FID,:]
    ax1.plot(xprofile[:],bed,linewidth=2.0,color=[139./255.,131./255.,120./255.])
    ax1.fill_between(xprofile[:], base, bed, where=bed >= base, facecolor=[1.0,0.9,0.6], interpolate=True)

    # Define top water surface  
    sea = water_lvl2[FID,:] + wave_height2[FID,:]
    ax1.plot(xprofile[:],sea,linewidth=2.0,color=[0./255.,0./255.,128./255.])
    ax1.fill_between(xprofile[:], bed, sea, where= sea > bed, facecolor=[0.7,0.9,1.0], interpolate=True)

    # Axes definitions
    tlt='XBeach model 2 time:'+str(int(times[FID]))+' s'
    plt.title(tlt, fontsize=12)
    ax1.set_ylabel('elevation [m]', fontsize=12)
    ax1.set_xlabel('lenght [m]', fontsize=12)
    plt.setp(ax1.get_xticklabels(), fontsize=10)
    plt.setp(ax1.get_yticklabels(), fontsize=10)
    
    # Set legend
    names=['zb','H+zs']
    ax1.legend(names,loc='lower right', fontsize=11)
    plt.show()

Now we can simply call the function above for different time steps:

In [ ]:
xbeach_frame1(1500)
xbeach_frame2(1500)

# Making an animation of the results

To visualise an animation of the water and profile evolution over the duration of the model we can use the following cell

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = plt.axes(xlim=(0, 5000), ylim=(-50, 20))
plt.title('XBeach model run', fontsize=12)
ax.set_ylabel('elevation [m]', fontsize=12)
ax.set_xlabel('lenght [m]', fontsize=12)
plt.setp(ax.get_xticklabels(), fontsize=10)
plt.setp(ax.get_yticklabels(), fontsize=10)
    
name = ['initial zb','zb','H+zs']

ax.plot(xprofile[:],bed_elev[0,:],'--', lw=1,color='k')

line1, = ax.plot([], [], lw=2,color=[139./255.,131./255.,120./255.] )
line2, = ax.plot([], [], lw=2,color=[0./255.,0./255.,128./255.])

# Initialization function: plot the background of each frame
def init():
    line1.set_data([], [])
    line2.set_data([], [])
    return line1,line2, 

base = -60.
top = np.zeros(len(xprofile[:]))
top.fill(20.)

# Animation function.  This is called sequentially
def animate(i):
    x = xprofile[:]
    f1 = bed_elev[i,:]
    sea = wave_height[i,:]+water_lvl[i,:]
    f2 = sea
    line1.set_data(x, f1)
    line2.set_data(x, f2)
    ax.fill_between(x, base, top, where= top > base, facecolor=[1.,1.,1.], interpolate=True)
    ax.fill_between(x, base, f1, where=f1 >= base, facecolor=[1.0,0.9,0.6], interpolate=True)
    ax.fill_between(x, f1, f2, where= f2 > f1, facecolor=[0.7,0.9,1.0], interpolate=True)
    ax.legend(name,loc='lower right', fontsize=9)
    return line1,line2 

# Call the animator.  blit=True means only re-draw the parts that have changed.
animation.FuncAnimation(fig, animate, init_func=init, frames=np.arange(0, 10000, 50), interval=200, blit=True)